In [1]:
from newspaper import Article
import newspaper
import os
import nafigator
import stanza
from nltk.corpus import stopwords
import pandas as pd

c:\anaconda3\envs\nlp\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
c:\anaconda3\envs\nlp\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator SelectKBest from version 0.19.0 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\anaconda3\envs\nlp\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator LogisticRegressionCV from version 0.19.0 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\anaconda3\envs\nlp\lib\site-pac

In [2]:
stopwords_nl = stopwords.words("dutch")
stopwords_en = stopwords.words("english")

In [3]:
def get_files(path):
    idx = 1
    list_of_files = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if (len(file)>4 and file[-10:].lower()=="index.html"): # or (file[-3:].lower()=="pdf"):
                list_of_files.append((idx, os.path.join(root,file)))
                idx += 1
    return list_of_files

In [4]:
list_of_files = get_files(path = "Z:\\websites\\obt\\www.dnb.nl")
print(len(list_of_files))

3211


In [5]:
nlp = {"nl": stanza.Pipeline('nl'), 
       "en": stanza.Pipeline('en')}

2022-02-07 08:24:09 INFO: Loading these models for language: nl (Dutch):
| Processor | Package |
-----------------------
| tokenize  | alpino  |
| pos       | alpino  |
| lemma     | alpino  |
| depparse  | alpino  |
| ner       | conll02 |

2022-02-07 08:24:09 INFO: Use device: cpu
2022-02-07 08:24:09 INFO: Loading: tokenize
2022-02-07 08:24:09 INFO: Loading: pos
2022-02-07 08:24:10 INFO: Loading: lemma
2022-02-07 08:24:10 INFO: Loading: depparse
2022-02-07 08:24:11 INFO: Loading: ner
2022-02-07 08:24:12 INFO: Done loading processors!
2022-02-07 08:24:12 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| ner       | ontonotes |

2022-02-07 08:24:12 INFO: Use device: cpu
2022-02-07 08:24:12 INFO: Loading: tokenize
2022-02-07 08:24:12 INFO: Loading: pos
2022-02-07 08:24:13 INFO: Loading: lemma
2022-02-07 08:24:14 INFO: Loading:

In [8]:
catalog = []

for idx, file in list_of_files:
    with open(file, 'r', encoding="utf-8") as fh:
        ht = fh.read()
    article = newspaper.Article("")
    article.set_html(ht)
    article.parse()

    if article.text != "":
        
        article_text = article.title + "\n\n" + article.text
        article_text_split = article_text.split(" ")

        print(repr(article.title))

        count_nl = 0
        for stopword in stopwords_nl:
            if stopword in article_text_split:
                count_nl += 1

        count_en = 0
        for stopword in stopwords_en:
            if stopword in article_text_split:
                count_en += 1

        if count_nl >= count_en:
            lang = "nl"
        else:
            lang = "en"

        txt_file = "..\\data\\websites\\www.dnb.nl-"+"{:05d}".format(idx)+".txt"
        naf_file = txt_file[:-4]+".naf.xml"

        if not os.path.isfile(naf_file):

            # store text in txt file
            with open(txt_file, 'w', encoding="utf-8") as f:
                f.write(article_text)
            f.close()
            print("txt written: "+txt_file)

            print("generating naf")
            doc = nafigator.generate_naf(
                input=txt_file,
                engine = "stanza",
                language=lang,
                naf_version = "v3.1",
                dtd_validation = False,
                params = {"public": {"uri": "https://"+file[20:-10]}},
                nlp=nlp[lang])
            doc.write(output=naf_file)
        else:
            print("already naffed: "+str(naf_file))
                    
        catalog.append([idx, "De Nederlandsche Bank", "https://"+file[20:-10], txt_file, lang, naf_file])
        
df_catalog = pd.DataFrame(data = catalog, columns = ['index', 'dc:creator', 'dc:relation', 'dc:source', 'dc:language', "naf:source"])

'De Nederlandsche Bank – de centrale bank van Nederland'
already naffed: ..\data\websites\www.dnb.nl-00001.naf.xml
'2020 in vogelvlucht'
already naffed: ..\data\websites\www.dnb.nl-00002.naf.xml
'Algemeen nieuws'
already naffed: ..\data\websites\www.dnb.nl-00003.naf.xml
'Blog Maarten Gelderman - Hoeveel mag toezicht kosten?'
already naffed: ..\data\websites\www.dnb.nl-00005.naf.xml
'Mag een toezichthouder bloggen?'
already naffed: ..\data\websites\www.dnb.nl-00006.naf.xml
'Blog Maarten Gelderman — Is ons toezicht te streng?'
already naffed: ..\data\websites\www.dnb.nl-00007.naf.xml
'Blog Maarten Gelderman - Mag een toezichthouder bloggen?'
already naffed: ..\data\websites\www.dnb.nl-00008.naf.xml
'Blog Maarten Gelderman - Toezichthouden, hoe werkt dat eigenlijk?'
already naffed: ..\data\websites\www.dnb.nl-00009.naf.xml
'Blog Olaf Sleijpen - Beter uit de crisis dankzij het welbegrepen eigenbelang'
already naffed: ..\data\websites\www.dnb.nl-00010.naf.xml
'Blog Olaf Sleijpen - Hoop doet

'“Opkomst BigTechs zorgt voor gebruiksgemak, maar er zijn ook risico’s”'
already naffed: ..\data\websites\www.dnb.nl-00085.naf.xml
'Tweefactorauthenticatie aankopen met credit card'
already naffed: ..\data\websites\www.dnb.nl-00087.naf.xml
'Update Maatschappelijk Verantwoord Beleggen'
already naffed: ..\data\websites\www.dnb.nl-00088.naf.xml
'Vijf vragen over de impact van de nieuwe duurzaamheidsstrategie'
already naffed: ..\data\websites\www.dnb.nl-00089.naf.xml
'20 jaar euro: 20 leuke weetjes'
already naffed: ..\data\websites\www.dnb.nl-00091.naf.xml
'Belangrijke stap naar toekomstbestendige arbeidsmarkt'
already naffed: ..\data\websites\www.dnb.nl-00092.naf.xml
'Betere communicatie gewenst over acceptatie contant betalen'
already naffed: ..\data\websites\www.dnb.nl-00093.naf.xml
'Betere Europese uitstootbeprijzing heeft beperkte gevolgen voor concurrentiepositie'
already naffed: ..\data\websites\www.dnb.nl-00094.naf.xml
"De sterke stijging van de inflatie in het eurogebied: transito

'“Materiaaltekorten hebben beperkte gevolgen voor Nederlandse economie”'
already naffed: ..\data\websites\www.dnb.nl-00155.naf.xml
'“Meer subsidies helpen starters niet”'
already naffed: ..\data\websites\www.dnb.nl-00156.naf.xml
'Minister publiceert evaluatie Toezicht, Resolutie en DGS van DNB'
already naffed: ..\data\websites\www.dnb.nl-00157.naf.xml
'Minister stuurt rapport Evaluatiecommissie Conservatrix naar Tweede Kamer'
already naffed: ..\data\websites\www.dnb.nl-00158.naf.xml
'Onthulling kunstinstallatie op Frederiksplein'
already naffed: ..\data\websites\www.dnb.nl-00159.naf.xml
'“Opkomst BigTechs zorgt voor gebruiksgemak, maar er zijn ook risico’s”'
already naffed: ..\data\websites\www.dnb.nl-00160.naf.xml
'Veelgestelde vragen over Conservatrix'
already naffed: ..\data\websites\www.dnb.nl-00161.naf.xml
'Vijf vragen over de impact van de nieuwe duurzaamheidsstrategie'
already naffed: ..\data\websites\www.dnb.nl-00162.naf.xml
'Wordt de beroepseed serieus genomen?'
already naffed

'Werkgroep Klimaatrisico: “bespreek als financiële instelling klimaatrisico’s met klanten”'
already naffed: ..\data\websites\www.dnb.nl-00223.naf.xml
'Daling aantal valse eurobiljetten'
already naffed: ..\data\websites\www.dnb.nl-00224.naf.xml
'DNB streeft naar nieuwe afspraken over contant geld'
already naffed: ..\data\websites\www.dnb.nl-00225.naf.xml
'Herstel in zicht na historische krimp in coronacrisis'
already naffed: ..\data\websites\www.dnb.nl-00226.naf.xml
'Raad van Bestuur van de ECB keurt zijn nieuwe monetairbeleidsstrategie goed'
already naffed: ..\data\websites\www.dnb.nl-00230.naf.xml
'Steven Maijoor treedt per 1 april 2021 toe tot directie DNB'
already naffed: ..\data\websites\www.dnb.nl-00231.naf.xml
'Stresstesten tonen weerbaarheid Europees bankensysteem'
already naffed: ..\data\websites\www.dnb.nl-00232.naf.xml
'Vlot economisch herstel verwacht na coronacrisis'
already naffed: ..\data\websites\www.dnb.nl-00233.naf.xml
'Vooruitzichten gunstig, maar kwetsbaarheden nemen

'DNB deelt uitkomsten enquête over transitie naar het nieuwe pensioenstelsel'
already naffed: ..\data\websites\www.dnb.nl-00302.naf.xml
'DNB houdt Beleidsregel maatschappelijke betamelijkheid trustkantoren ongewijzigd aan'
already naffed: ..\data\websites\www.dnb.nl-00303.naf.xml
'DNB kiest voor proportionele invulling RSR 2021 (in te sturen in 2022)'
already naffed: ..\data\websites\www.dnb.nl-00304.naf.xml
'DNB organiseert ronde tafel over beroepseed'
already naffed: ..\data\websites\www.dnb.nl-00305.naf.xml
'DNB publiceert toezichtplannen voor 2022'
already naffed: ..\data\websites\www.dnb.nl-00306.naf.xml
'ECB start met harmonisatie van statistiekrapportages'
already naffed: ..\data\websites\www.dnb.nl-00307.naf.xml
'FATF waarschuwingslijsten update juni 2021'
already naffed: ..\data\websites\www.dnb.nl-00308.naf.xml
'FATF waarschuwingslijsten – update maart 2021'
already naffed: ..\data\websites\www.dnb.nl-00309.naf.xml
'FATF waarschuwingslijsten – update oktober 2021'
already naf

...

'Besluitvormingsproces opdrachtaanvaarding premieovereenkomst door pensioenfonds'
already naffed: ..\data\websites\www.dnb.nl-02833.naf.xml
'Combinatie vaste daling en het toepassen van een spreidingsperiode bij een variabele pensioenuitkering'
already naffed: ..\data\websites\www.dnb.nl-02834.naf.xml
'Mag de risicohouding voor een premieregeling of een variabele uitkering per leeftijdscohort worden vastgelegd?'
already naffed: ..\data\websites\www.dnb.nl-02835.naf.xml
'Kan een pensioenfonds dat zowel een uitkeringsregeling als een premieregeling uitvoert hiervoor verschillende risicohoudingen vastleggen?'
already naffed: ..\data\websites\www.dnb.nl-02836.naf.xml
'Regels beleggingen voor variabele uitkeringen'
already naffed: ..\data\websites\www.dnb.nl-02837.naf.xml
'Passendheid toegepaste rekenmethodiek'
already naffed: ..\data\websites\www.dnb.nl-02838.naf.xml
'Actuariële en bedrijfstechnische nota'
already naffed: ..\data\websites\www.dnb.nl-02839.naf.xml
'Beleidsregel gedragscode 

'Openbaar register premiepensioeninstelling'
already naffed: ..\data\websites\www.dnb.nl-02909.naf.xml
'Proces vergunningaanvraag (10 stappen) premiepensioeninstelling'
already naffed: ..\data\websites\www.dnb.nl-02910.naf.xml
'Q&A’s premiepensioeninstelling'
already naffed: ..\data\websites\www.dnb.nl-02911.naf.xml
'Succesfactoren vergunningaanvraag premiepensioeninstelling'
already naffed: ..\data\websites\www.dnb.nl-02912.naf.xml
'Vereisten voor een vergunning van een premiepensioeninstelling'
already naffed: ..\data\websites\www.dnb.nl-02913.naf.xml
'Vergunningplicht voor premiepensioeninstelling (PPI)'
already naffed: ..\data\websites\www.dnb.nl-02914.naf.xml
'Wettelijk kader markttoegang premiepensioeninstelling'
already naffed: ..\data\websites\www.dnb.nl-02915.naf.xml
'Integriteit en trustkantoren'
already naffed: ..\data\websites\www.dnb.nl-02917.naf.xml
'DNB publiceert Beleidsregel maatschappelijke betamelijkheid voor trustkantoren'
already naffed: ..\data\websites\www.dnb.nl

'Kapitaal'
txt written: ..\data\websites\www.dnb.nl-02991.txt
generating naf
'Pilaar 1: Aanvullend Eigen Vermogen'
txt written: ..\data\websites\www.dnb.nl-02992.txt
generating naf
'Pilaar 1: Eigen Vermogensbestanddelen die niet zijn opgenomen op de lijst'
txt written: ..\data\websites\www.dnb.nl-02993.txt
generating naf
'Pilaar 1: Matchingopslag'
txt written: ..\data\websites\www.dnb.nl-02994.txt
generating naf
'Pilaar 1: Ondernemingsspecifieke Parameters'
txt written: ..\data\websites\www.dnb.nl-02995.txt
generating naf
'Q&A Deferral Adjustment'
txt written: ..\data\websites\www.dnb.nl-02996.txt
generating naf
'Q&A en Good practices over de behandeling spaarhypotheken onder Solvency II'
txt written: ..\data\websites\www.dnb.nl-02997.txt
generating naf
'Q&A erkenning herverzekeringscontracten als risicolimiteringstechnieken in de Solvency II StandaardFormule'
txt written: ..\data\websites\www.dnb.nl-02998.txt
generating naf
'Q&A Kosten in technische voorzieningen van verzekeraars'
txt

'Wisselinstellingen – Register'
txt written: ..\data\websites\www.dnb.nl-03067.txt
generating naf
'Wisselinstellingen – Termijn voor behandeling vergunningaanvraag'
txt written: ..\data\websites\www.dnb.nl-03068.txt
generating naf
'Wisselinstellingen – Vragen over de aanvraag van een vergunning'
txt written: ..\data\websites\www.dnb.nl-03069.txt
generating naf
'Beleidsregel geschiktheid 2012'
txt written: ..\data\websites\www.dnb.nl-03070.txt
generating naf
'Convenant DNB - AFM'
txt written: ..\data\websites\www.dnb.nl-03071.txt
generating naf
'De werking van Europese regelgeving'
txt written: ..\data\websites\www.dnb.nl-03072.txt
generating naf
'DNB Toezicht uit hoofde van Securitisation Regulation'
txt written: ..\data\websites\www.dnb.nl-03073.txt
generating naf
'Europees Financieel Toezicht'
txt written: ..\data\websites\www.dnb.nl-03074.txt
generating naf
'Protocol tussen De Nederlandsche Bank N.V. en de Nederlandse Mededingingsautoriteit betreffende concentraties in de financiële

'Opvolging aanbevelingen European Banking Authority aan toezichthouders ten aanzien van overgang van PSD1 naar PSD2'
txt written: ..\data\websites\www.dnb.nl-03136.txt
generating naf
'Procedure voor het melden van een uitzondering'
txt written: ..\data\websites\www.dnb.nl-03137.txt
generating naf
'Proces om als betaalinitiatiedienstaanbieder actief te worden'
txt written: ..\data\websites\www.dnb.nl-03138.txt
generating naf
'Proces voor rekeninginformatiedienstaanbieder om actief te worden'
txt written: ..\data\websites\www.dnb.nl-03139.txt
generating naf
'Q&A Veilige zakelijke betalingsprocedures en -protocollen'
txt written: ..\data\websites\www.dnb.nl-03140.txt
generating naf
'Reikwijdte betaaldienstverlening - niet op zichzelf staande betaaltransacties'
txt written: ..\data\websites\www.dnb.nl-03141.txt
generating naf
'Rekeninginformatiedienstverlening en andere bedrijfsactiviteiten'
txt written: ..\data\websites\www.dnb.nl-03142.txt
generating naf
'Spaarrekeningen'
txt written: ..

In [10]:
df_catalog.to_csv("..//data//websites//catalog.csv")

In [ ]:
naf_file